In [17]:
import yaml
import sqlparse
import os
import re

import pandas as pd
os.chdir('z:/Covid19/Covid19_OMOP/new_pipeline')
print(os.getcwd())

from omop_etl.datastore import DataStore
from omop_etl.load import Loader
from omop_etl.utils import search
from omop_etl.bo import bo_query, format_stage_query

omop = DataStore('config.yml')

z:\Covid19\Covid19_OMOP\new_pipeline


# Configuration files

In [18]:
from omop_etl.config import ProjectConfig, ETLConfig

In [19]:
config = ProjectConfig('config.yml')

In [20]:
# config._config.get('load')

In [21]:
# config = ProjectConfig('config.yml')

In [22]:
# with config.engine.connect() as con:
#     print(con)

In [23]:
etl_config = ETLConfig()

# BO Queries

In [24]:
omop = DataStore('config.yml')

# def format_stage_query(dp_name):

#     with open('omop_etl/etl_config.yml') as f:
#         yml = yaml.safe_load(f) 
    
#     aliases= yml['aliases'][dp_name]
#     bo_queries = omop.get_bo_query('omop')

#     start_date = omop.config_param['date_range']['start_date']
#     end_date = omop.config_param['date_range']['end_date']

#     sql_query = format_bo_sql(bo_queries[dp_name], dp_name, schema='stage', aliases=aliases)
#     patient_id = "select PATIENT_KEY from DWS_OMOP.cohort.PersonList"
#     sql_query = sql_query.replace("12345678", patient_id)
#     sql_query = sql_query.replace("01/01/1900 00:0:0", start_date)
#     sql_query = sql_query.replace("12/31/1900 00:0:0", end_date)

#     sql_query = sqlparse.format(sql_query, reindent_aligned=True, indent_with=1)

#     return f"EXECUTE ('USE [DWS_PROD];\n {sql_query}')" 

## BO cohort queries

In [25]:
from omop_etl.bo import bo_query
with omop.engine.connect() as con:
    bo_queries = bo_query('omop', con)

len(bo_queries.keys())
# [q for q in bo_queries.keys() if bo_queries[q] == '']

60

In [26]:
for dp in bo_queries.keys():
    if len(bo_queries[dp]) == 0:
        print(dp)

In [27]:
loader = Loader('config.yml')

In [29]:
print(loader.stage_table('measurement', subset='res_tidal_spont', only_query=True))

Executing stage_table(measurement, ['res_tidal_spont', True]) ... Done   
Elapsed time 00:00:02
EXECUTE ('USE DWS_PROD;
 DROP TABLE IF EXISTS dws_omop.stage.MEASUREMENT_Res_Tidal_Spont;SELECT  dbo.ALL_PATIENTS.PATNT_KEY AS patient_key, PATNT_ENCNTR_KEY_XREF1.PATNT_ENCNTR_KEY AS patnt_encntr_key, Flowsheet_Enc_Dttime_RESP.MEASR_TAKN_DATE AS respiratory_date, Flowsheet_Enc_Dttime_RESP.MEASR_TAKN_DT AS respiratory_datetime, Flowsheet_RESP_TV_SPONT.MEASR_VALUE AS tidal_volume_exhaled, ALL_PROVIDER_IDS_ATTEND_SER.PROVIDR_KEY AS attending_provider, ALL_PROVIDERS_VISIT.PROVIDR_KEY AS visit_provider INTO dws_omop.stage.MEASUREMENT_Res_Tidal_Spont  FROM  dbo.ALL_PATIENTS RIGHT OUTER JOIN dbo.ALL_PATIENT_SNAPSHOTS ALL_PT_SNAPSHOTS_ENCOUNTER ON (ALL_PT_SNAPSHOTS_ENCOUNTER.PATNT_KEY=dbo.ALL_PATIENTS.PATNT_KEY)  RIGHT OUTER JOIN dbo.PATIENT_ENCOUNTER_DTL ON (ALL_PT_SNAPSHOTS_ENCOUNTER.PATNT_SNAPSHT_KEY=dbo.PATIENT_ENCOUNTER_DTL.DSCHRG_PATNT_SNAPSHT_KEY AND dbo.PATIENT_ENCOUNTER_DTL.TEST_IND=''N''
 

In [13]:
from omop_etl.bo import bo_query
with omop.engine.connect() as con:
    bo_cohort = bo_query('cohort_COVID_broad', con)

table = 'CPT_CodingDetail'
# print(format_stage_query(table))
bo_cohort[table]

"SELECT  PX_ALL_PATIENTS.PATNT_KEY,  PX_ALL_PATIENTS.PATNT_ID,  PX_ALL_PATIENT_IDENTITIES.IDENT_ID_INT,  PX_JAX_MRN.IDENT_ID_INT FROM  ALL_PATIENTS PX_ALL_PATIENTS RIGHT OUTER JOIN ALL_PATIENT_SNAPSHOTS PX_ALL_PATIENT_SNAPSHOTS ON (PX_ALL_PATIENT_SNAPSHOTS.PATNT_KEY=PX_ALL_PATIENTS.PATNT_KEY)  LEFT OUTER JOIN ALL_PATIENT_IDENTITIES PX_ALL_PATIENT_IDENTITIES ON (PX_ALL_PATIENT_SNAPSHOTS.PATNT_KEY=PX_ALL_PATIENT_IDENTITIES.PATNT_KEY and PX_ALL_PATIENT_IDENTITIES.LOOKUP_IND = 'Y' and PX_ALL_PATIENT_IDENTITIES.IDENT_ID_TYPE = 101)  LEFT OUTER JOIN ALL_PATIENT_IDENTITIES PX_JAX_MRN ON (PX_ALL_PATIENT_SNAPSHOTS.PATNT_KEY=PX_JAX_MRN.PATNT_KEY and PX_JAX_MRN.LOOKUP_IND = 'Y' and PX_JAX_MRN.IDENT_ID_TYPE = 110)  RIGHT OUTER JOIN PROCEDURE_EVENT_DTL ON (PROCEDURE_EVENT_DTL.PATNT_SNAPSHT_KEY=PX_ALL_PATIENT_SNAPSHOTS.PATNT_SNAPSHT_KEY)  LEFT OUTER JOIN ALL_CPT_PROCEDURE_CODES ON (PROCEDURE_EVENT_DTL.CPT_CD_KEY=ALL_CPT_PROCEDURE_CODES.CPT_CD_KEY)  WHERE  (  PROCEDURE_EVENT_DTL.START_DATE BETWEEN '0

In [14]:
from omop_etl.bo import bo_query, format_stage_query
import sqlparse

In [15]:
with omop.engine.connect() as con:
    print(format_stage_query('omop', 'MEASUREMENT_Res_RESP_MechVentSetRate', '01/01/2012', '12/31/2020', con, schema='stage'))

EXECUTE ('USE DWS_PROD;
 DROP TABLE IF EXISTS dws_omop.stage.MEASUREMENT_Res_RESP_MechVentSetRate;SELECT DISTINCT  dbo.ALL_PATIENTS.PATNT_KEY, PATNT_ENCNTR_KEY_XREF1.PATNT_ENCNTR_KEY, Flowsheet_Enc_Dttime_RESP.MEASR_TAKN_DATE, Flowsheet_Enc_Dttime_RESP.MEASR_TAKN_DT, Flowsheet_RESP_Mech_Vent.MEASR_VALUE, ALL_PROVIDER_IDS_ATTEND_SER.PROVIDR_KEY AS ALL_PRO_IDS_ATT_SER_PROVIDR_KEY, ALL_PROVIDERS_VISIT.PROVIDR_KEY AS ALL_PRO_VIS_PROVIDR_KEY INTO dws_omop.stage.MEASUREMENT_Res_RESP_MechVentSetRate  FROM  dbo.ALL_PATIENTS RIGHT OUTER JOIN dbo.ALL_PATIENT_SNAPSHOTS ALL_PT_SNAPSHOTS_ENCOUNTER ON (ALL_PT_SNAPSHOTS_ENCOUNTER.PATNT_KEY=dbo.ALL_PATIENTS.PATNT_KEY)  RIGHT OUTER JOIN dbo.PATIENT_ENCOUNTER_DTL ON (ALL_PT_SNAPSHOTS_ENCOUNTER.PATNT_SNAPSHT_KEY=dbo.PATIENT_ENCOUNTER_DTL.DSCHRG_PATNT_SNAPSHT_KEY AND dbo.PATIENT_ENCOUNTER_DTL.TEST_IND=''N''
 )  LEFT OUTER JOIN dbo.ALL_PROVIDERS ALL_PROVIDERS_ATTENDING ON (dbo.PATIENT_ENCOUNTER_DTL.ATTND_PROVIDR_KEY=ALL_PROVIDERS_ATTENDING.PROVIDR_KEY AND 

In [18]:
# bo['MEASUREMENT_Res_RESP_MechVentSetRate']

In [ ]:
# with omop.engine.connect() as con:
#     bo_queries = bo_query('omop', con)
# # print(bo_queries.keys())

# # SAVE QUERIES TO FILE
# for t in bo_queries.keys():
#     try:
#         print(t)
#         f = open(f'./output/sqlstring_{t.lower()}.sql', 'w')
#         f.write(format_stage_query(t))
#         format_stage_query(t)
#         f.close()
#     except ValueError as e:
#         print(t)
#         raise e

# aliases= yml[table]
# format_bo_sql(bo_queries[table], table, schema='stage')
# omop.execute(format_stage_query('MEASUREMENT_HeartRate'))

# aliases = {}
# for t in bo_queries.keys():
#     with omop.engine.connect() as con:
#         cols = pd.read_sql(f"select top 0 * from stage.{t}", con)
#         aliases[t] = [x.lower() for x in cols.columns]

# with open('col_aliases.yml', 'w') as f:
#     yaml.dump(aliases, f)
#     f.close()

In [ ]:
bo_queries = omop.get_bo_query('omop')
sqlstring = bo_queries['MEASUREMENT_BP_CVP']#'MEASUREMENT_BP_MAP_Cuff'] or 'MEASUREMENT_Height' MEASUREMENT_Weight
parsed = sqlparse.parse(sqlstring)[0]
idx = [parsed.token_index(t) for t in parsed if t.is_keyword and t.value == 'FROM'][0]
columns = parsed.token_prev(idx)[1]

# Extract columns from SELECT clause. If duplicated columns, use alias, 
# else append abbreviated source table name.
colnames = [i.value.split('.')[-1] for i in columns]
dup_cols = set([x for x in colnames if colnames.count(x) > 1])
new_colnames = []

counter = 0
for item in columns:
    if isinstance(item, sqlparse.sql.Identifier):
        colname = item.value.split('.')[-1]
        tabname = item.value.split('.')[-2]
        shrt_tabname = '_'.join([word[:3] for word in tabname.split('_')])
        # print('Identifier', item.value)
    
    elif isinstance(item, sqlparse.sql.Function):
        counter += 1
        fn_name = f'FN_{counter}'
        for x in item:
            if (isinstance(x, sqlparse.sql.Identifier) and x.value == 'cast'):
                print(item.value)
                continue
            else: 
                pass
            if isinstance(x, sqlparse.sql.Parenthesis): 
                for y in x:
                    if isinstance(y, sqlparse.sql.Identifier):
                        print(y.value)
                        print([i.strip() for i in y.value.split(' as ')])
                        # col, dtype = [i.strip() for i in y.value.split('as')]
                        # print(f'try_convert({dtype}, {col})')
    
    # else: 
    #     print('Something else', item.__repr__)

In [ ]:
def flatten(alist):
    flat = []
    while alist: 
        e = alist.pop()
        if type(e) == list: 
            alist.extend(e) 
        else:
            flat.append(e) 
     
    return flat

# def replace_cast_with_try_convert(parsed):    
items = []
for token in parsed.tokens:
    if isinstance(token, sqlparse.sql.IdentifierList) or isinstance(token, sqlparse.sql.Where):
        items = [item for item in token if search('cast', item.value)]
        # print(items)
        for item in items:
            if isinstance(item, sqlparse.sql.Function):
                
                col, dtype = flatten([[i.value.split(' as ') for i in p if isinstance(i, sqlparse.sql.Identifier)] 
                                       for p in item if isinstance(p, sqlparse.sql.Parenthesis)])
                item.value = f'try_convert({dtype},{col})'
                
            else:
                fun_list = flatten([[a for a in i if isinstance(a, sqlparse.sql.Function)] 
                                     for i in item if search('cast', i.value)])
                
                # print(fun_list)
                if not fun_list:
                    fun_list = [i for i in item if (search('cast', i.value) & isinstance(i, sqlparse.sql.Function))]
                
                if not fun_list:
                    fun_list = flatten(flatten([[[a for a in b if isinstance(a, sqlparse.sql.Function)] 
                                                for b in i if isinstance(b, sqlparse.sql.Operation)] for i in item if search('cast', i.value)]))

                # print(fun_list)
                for fun in fun_list:
                    col, dtype = flatten([[i.value.split(' as ') for i in p if isinstance(i, sqlparse.sql.Identifier)] 
                                           for p in fun if isinstance(p, sqlparse.sql.Parenthesis)][0])
                    # print(fun)
                    item.value = item.value.replace(fun.value, f'try_convert({dtype},{col})')
                    # print(item.value)
        
        token.value = ''.join(item.value for item in token)

        print(token.value)

In [ ]:
import numpy as np

bo_queries = omop.get_bo_query('omop')
sqlstring = bo_queries['MEASUREMENT_Height']#'MEASUREMENT_BP_MAP_Cuff'] or 'MEASUREMENT_Height' MEASUREMENT_Weight MEASUREMENT_BP_CVP
parsed = sqlparse.parse(sqlstring)[0]

def get_function(item):
    if isinstance(item, (sqlparse.sql.Parenthesis, sqlparse.sql.Operation)):
        return list(filter(None, [get_function(a) for a in item]))
    elif isinstance(item, sqlparse.sql.Function):
        return item

def flatten(lst):
    for el in lst:
        if isinstance(el, list):  
            # recurse
            yield from flatten(el)
        else:
            # generate
            yield el
 

# fun_list = []
for token in parsed.tokens:
    if isinstance(token, (sqlparse.sql.IdentifierList, sqlparse.sql.Where)):
        items = [item for item in token if search('cast', item.value)]
        # print(items)
        for item in items:
            if isinstance(item, sqlparse.sql.Function):
                fun_list = flatten([get_function(item)])
                # print(fun_list)

            else:
                fun_list = flatten(list(filter(None, [get_function(i) for i in item])))
                # print(fun_list)

            for fun in fun_list:
                col, dtype = flatten([[i.value.split(' as ') for i in p if isinstance(i, sqlparse.sql.Identifier)] 
                                        for p in fun if isinstance(p, sqlparse.sql.Parenthesis)][0])
                # print(fun)
                item.value = item.value.replace(fun.value, f'try_convert({dtype},{col})')
    
        token.value = ''.join(item.value for item in token)

        print(token.value)

In [ ]:
from functools import reduce

In [ ]:
reduce(list.__add__, (list(i) for i in [[i.value.split(' as ') for i in p if isinstance(i, sqlparse.sql.Identifier)] 
                                        for p in fun if isinstance(p, sqlparse.sql.Parenthesis)][0]))

In [ ]:
fun_list

In [ ]:
for token in parsed.tokens:
#     print(token.ttype)
    if isinstance(token, sqlparse.sql.IdentifierList) or isinstance(token, sqlparse.sql.Where):
        print(token.ttype)

In [ ]:
from omop_etl.stage import Stager
stage = Stager('config.yml')
# print(l.update_mappings('person'))
# print(l.update_mappings('visit_occurrence'))

# print(l.preload_all())

# print(l.load_table('person'))
# print(l.load_table('death'))
# print(l.load_table('condition_occurrence'))
# print(l.load_table('procedure_occurrence'))
# print(l.load_table('drug_exposure'))
# print(l.load_table('observation'))
# print(l.load_table('provider'))
# print(l.load_table('care_site'))
# print(l.load_table('location'))

In [ ]:
load_conf = stage.store.config_param['load']
# stage.stage_table('measurement','res_fio2')

In [ ]:
for t in load_conf.keys():
    if load_conf[t]:
        for part in load_conf[t].keys():
            stage.stage_table(t, part)
    else:
        if t not in ('provider','care_site','location'): 
            stage.stage_table(t)

In [ ]:
import itertools
import re 

for token in parsed.tokens:
    if isinstance(token, sqlparse.sql.Where):
        print([item for item in token if search('cast', item.value)])

        # par = list(itertools.chain(*token))
        # for p in par:
        #     print(p.__repr__)
            # if isinstance(i, sqlparse.sql.Parenthesis):
            #     # defs = extract_definitions(i)
            #     for x in i:
            #         if isinstance(x, sqlparse.sql.Parenthesis):
            #             for y in x:
            #                 print(y.__repr__)

In [ ]:
import sqlparse

def extract_definitions(token_list):
    # assumes that token_list is a parenthesis
    definitions = []
    tmp = []
    par_level = 0
    for token in token_list.flatten():
        if token.is_whitespace:
            continue
        elif token.match(sqlparse.tokens.Punctuation, '('):
            par_level += 1
            continue
        if token.match(sqlparse.tokens.Punctuation, ')'):
            if par_level == 0:
                break
            else:
                par_level += 1
        elif token.match(sqlparse.tokens.Punctuation, ','):
            if tmp:
                definitions.append(tmp)
            tmp = []
        else:
            tmp.append(token)
    if tmp:
        definitions.append(tmp)
    return definitions

# Check config files

- Check tables match in config and etl_config
- Check subset names match in config and etl_config
- Check dp_names match bo dp_names
- Check scripts in etl_config exists
- Check all stage tables have aliases
- Check aliases match stage table columns
- Check load tables match table and subset names
- Check load scripts exists
- Check table name with source_table
- Check source_code from table match source_code from query

In [1]:
import os
import glob
import pandas as pd

os.chdir('Z:/OMOP/omop_etl/omop_etl/templates')
print(os.getcwd())

from omop_etl.datastore import DataStore
from omop_etl.bo import bo_query
from omop_etl.config import ProjectConfig, ETLConfig

Z:\OMOP\omop_etl\omop_etl\templates


In [2]:
database = 'dws_omop'
server = 'edw.shands.ufl.edu'

omop = DataStore('config.yml')
omop.connection_str = f'mssql+pyodbc://{server}/{database}?driver=SQL+Server'

with omop.engine.connect() as con:
    bo_queries = bo_query('omop', con)

_config = ProjectConfig('config.yml')
etl_config = ETLConfig()
sql_scripts = [os.path.split(p)[1] for p in glob.glob('Z:/OMOP/omop_etl/omop_etl/sql/*.sql') ]

path = 'Z:/OMOP/omop_etl/omop_etl/sql'

In [3]:
# sql_scripts

In [4]:
config_load = _config.load
etl_stage = etl_config.stage
etl_preload = etl_config.preload
etl_mapping = etl_config.mapping
etl_load = etl_config.preload

In [5]:
def file_exists(file):
    path = 'Z:/OMOP/omop_etl/omop_etl/sql'
    filepath = os.path.join(path, file)
    
    return os.path.exists(filepath)

In [6]:
dp_names = []
preload_scripts = []
mapping_scripts = []
load_scripts = []

for table in config_load.keys():
    if config_load[table]:
        for sbs in config_load[table]:
            stg = etl_stage[table]
            dp_names.append(etl_stage[table][sbs] if isinstance(stg, dict) else stg)
            load_scripts.append(etl_load[table][sbs] if isinstance(stg, dict) else stg)
            
            if isinstance(stg, dict):
                print(f'{table}, {sbs} load script exists?', file_exists(etl_load[table][sbs]))
            
            if table in etl_preload.keys():
                preload_scripts.append(etl_preload[table][sbs])
                print(f'{table}, {sbs} preload script exists?', file_exists(etl_preload[table][sbs]))
                
    else:
        if table in etl_stage.keys():
            dp_names.append(etl_stage[table])
        if table in etl_mapping.keys():
            mapping_scripts.append(etl_mapping[table])
            print(f'{table} mapping script exists?', file_exists(etl_mapping[table]))

person mapping script exists? True
visit_occurrence mapping script exists? True
procedure_occurrence, icd load script exists? True
procedure_occurrence, icd preload script exists? True
procedure_occurrence, cpt load script exists? True
procedure_occurrence, cpt preload script exists? True
drug_exposure, order load script exists? True
drug_exposure, order preload script exists? True
drug_exposure, admin load script exists? True
drug_exposure, admin preload script exists? True
measurement, bp load script exists? True
measurement, bp preload script exists? True
measurement, bp_artline load script exists? True
measurement, bp_artline preload script exists? True
measurement, bp_noninvasive load script exists? True
measurement, bp_noninvasive preload script exists? True
measurement, bp_cvp load script exists? True
measurement, bp_cvp preload script exists? True
measurement, bp_cvp_mean load script exists? True
measurement, bp_cvp_mean preload script exists? True
measurement, bp_map_artline l

In [7]:
set(dp_names) == set(bo_queries.keys())

False

In [8]:
# Wrong dp_name in etl_config
set(dp_names) - set(bo_queries.keys())

set()

In [9]:
# dp_names missing in etl_config
set(bo_queries.keys()) - set(dp_names)

{'MEASUREMENT_Res_RESP_AdultMech', 'MEASUREMENT_Res_RESP_PedsMech'}

In [10]:
# Check if preload tables exist in config, etl stage, and etl load
for prl in etl_preload:
    if prl in config_load.keys():
        print(f'Preload table {prl} found in config.')
    if prl in etl_stage.keys():
        print(f'Preload table {prl} found in etl_stage')
    if prl in etl_load.keys():
        print(f'Preload table {prl} found in etl_load')

Preload table condition_occurrence found in config.
Preload table condition_occurrence found in etl_stage
Preload table condition_occurrence found in etl_load
Preload table drug_exposure found in config.
Preload table drug_exposure found in etl_stage
Preload table drug_exposure found in etl_load
Preload table measurement found in config.
Preload table measurement found in etl_stage
Preload table measurement found in etl_load
Preload table observation found in config.
Preload table observation found in etl_stage
Preload table observation found in etl_load
Preload table procedure_occurrence found in config.
Preload table procedure_occurrence found in etl_stage
Preload table procedure_occurrence found in etl_load


In [11]:
temp_path = 'Z:/OMOP/omop_etl/omop_etl/templates'

In [12]:
source = pd.read_csv(os.path.join(temp_path, 'source_to_concept_map.csv'), sep='\t')

In [13]:
source

,source_code,source_concept_id,source_vocabulary_id,source_code_description,target_concept_id,target_vocabulary_id,valid_start_date,valid_end_date
0,?,0,Admit Source,ALL_ADMIT_SOURCES.STNDRD_LABEL,0,Visit,2020-09-29,2099-12-31
1,EXTRAMURAL BIRTH,0,Admit Source,ALL_ADMIT_SOURCES.STNDRD_LABEL,0,Visit,2020-09-29,2099-12-31
2,FROM DISASTER SITE,0,Admit Source,ALL_ADMIT_SOURCES.STNDRD_LABEL,0,Visit,2020-09-29,2099-12-31
3,HMO REFERRAL,0,Admit Source,ALL_ADMIT_SOURCES.STNDRD_LABEL,0,Visit,2020-09-29,2099-12-31
4,NON HEALTHCARE FACILITY,0,Admit Source,ALL_ADMIT_SOURCES.STNDRD_LABEL,0,Visit,2020-09-29,2099-12-31
...,...,...,...,...,...,...,...,...
162,NONBINARY,8570,Sex,ALL_SEXES.STNDRD_LABEL,0,Gender,2020-09-29,2099-12-31
163,OTHER,8521,Sex,ALL_SEXES.STNDRD_LABEL,0,Gender,2020-09-29,2099-12-31
164,UNKNOWN,8551,Sex,ALL_SEXES.STNDRD_LABEL,0,Gender,2020-09-29,2099-12-31
165,MALE,8507,Sex,ALL_SEXES.STNDRD_LABEL,8507,Gender,2020-09-29,2099-12-31
